In [ ]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import cv2

In [ ]:
def visualize_map(train_waypoint, train_df,dpi=100):
    map_floor = {'B2': -2, 'B1': -1, 'F1': 0, 'F2': 1, 'F3': 2, 'F4': 3, 'F5': 4, 'F6': 5, 'F7': 6, 'F8': 7, 'F9': 8,
                 '2B': -2, '1B': -1, '1F': 0, '2F': 1, '3F': 2, '4F': 3, '5F': 4, '6F': 5, '7F': 6, '8F': 7, '9F': 8}
 
    site_list = train_waypoint['site'].unique()

    # site loop
    for site in sorted(site_list):
        floor_list = sorted(glob(f'{DIR_META}{site}/*'))
        floor_list = [text.split('/')[-1] for text in floor_list]
        
        # 画像の比を決めるために一度floor_info.jsonを取得
        path_json = f'{DIR_META}{site}/{floor_list[0]}/floor_info.json'
        with open(path_json) as json_file:
            json_data = json.load(json_file)
        _width_meter = json_data["map_info"]["width"]
        _height_meter = json_data["map_info"]["height"]
        s = max(_width_meter, _height_meter)

        t_site_wp = train_waypoint.loc[train_waypoint['site']==site, :].reset_index(drop=True)
        t_site_wp2 = train_df.loc[train_df['site']==site, :].reset_index(drop=True)
        
        # figとaxesの用意
        n_floor = len(floor_list)
        n_row = int(np.ceil(n_floor / 3))
        w = round(5*3*_width_meter/s)
        h = round(5*n_row*_height_meter/s)
        fig, axs = plt.subplots(n_row, 3, figsize=(w, h), dpi=dpi)
        # fig.suptitle(f'{site} (train:{len(t_site_wp)} extra:{len(a_site_wp)})', fontsize=15)
        fig.suptitle(site, fontsize=15)
        fig.patch.set_facecolor('white')
        axs = axs.ravel()

        # floor loop
        for i_floor, floor in enumerate(floor_list):
            # json
            path_json = f'{DIR_META}{site}/{floor}/floor_info.json'
            with open(path_json) as json_file:
                json_data = json.load(json_file)
            width_meter = json_data["map_info"]["width"]
            height_meter = json_data["map_info"]["height"]

            # show image
            path_img = f'{DIR_META}{site}/{floor}/floor_image.png'
            im = Image.open(path_img)
            axs[i_floor].imshow(np.asarray(im), extent=(0, width_meter, 0, height_meter))
            
            t_wp = t_site_wp.loc[t_site_wp['floor']==map_floor[floor], :].reset_index(drop=True)
            t_wp2 = t_site_wp2.loc[t_site_wp2['floor']==map_floor[floor], :].reset_index(drop=True)

            # show waypoint
            axs[i_floor].scatter(t_wp['x'], t_wp['y'], marker='o', color='black', s=1, label="Given")
            t_wp2 = t_wp2[5>t_wp2['distance']]
            axs[i_floor].scatter(t_wp2['ix'], t_wp2['iy'], marker='o', color='green', s=0.5, alpha=0.5, label="LI")
            axs[i_floor].scatter(t_wp2['fx'], t_wp2['fy'], marker='o', color='orange', s=0.5, alpha=0.5, label="KF")
            
#             for path, path_data in t_wp2.groupby("path"):
#                 path_data.plot(x="ix", y="iy", style=".-", ax=axs[i_floor], alpha=0.5, color='orange', markersize=1, linewidth=0.5, legend=False)
#             for path, path_data in t_wp2.groupby("path"):
#                 path_data.plot(x="fx", y="fy", style=".-", ax=axs[i_floor], alpha=0.5,color='black',  markersize=1, linewidth=0.5, legend=False)
                
            axs[i_floor].set_title(floor, fontsize=10)
            
            
            axs[i_floor].legend()
        plt.tight_layout()
        plt.show()
        break

In [ ]:
DIR_META = "../input/indoor-location-navigation/metadata/"

train_waypoint = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')

In [ ]:
import pickle
def from_pickle(filename):
    with open(filename, mode='rb') as f:
        obj = pickle.load(f)
    return obj

train_df = from_pickle('../input/2kaido_wifi_dataset_v5/train_10000_7.pkl')
train_df

In [ ]:
train_df['timestamp'] = train_df['timestamp'].astype(int) - train_df['timediff'].astype(int)
train_df = train_df[['site_id', 'path', 'timestamp','floor','ix','iy','x','y','fx','fy']]
train_df = train_df.astype({'timestamp':int, 'floor':np.float32,'x': np.float32, 'y': np.float32,'ix':np.float32, 'iy':np.float32,'fx':np.float32, 'fy':np.float32})
train_df = train_df.rename(columns={'site_id':'site'})
train_df['distance'] = np.sqrt((train_df['ix']-train_df['fx'])**2 + (train_df['iy']-train_df['fy'])**2)
train_df

In [ ]:
train_df['distance'].hist(bins=50)
plt.xlabel('distance betweeen linear waypoint and kalman waypoint')

In [ ]:
visualize_map(train_waypoint, train_df)